# ML Foreclosure Binary Classification - 5% sample dataset
Only "resolved" outcomes - Either paid off or default 

    target = zeroBalCode (set to object dtype) 
    Binary classification sets zeroBalCode to either "0" or "1" for the 7 possible outcomes
    
    "0" = "Loan Success"
        * 01 = Prepaid or Matured
        * 06 = Repurchased
        * 16 = Reperforming Loan Sale

    "1" =  "Loan Failure" - Most likely loss of money (but not necessarily) one of below conditions:
        * 02 = Third Party Sale
        * 03 = Short Sale
        * 09 = Deed-in-Lieu,REO
        * 15 = Note Sale


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#!pip install pycaret
from imblearn.over_sampling import RandomOverSampler
from pycaret.classification import *
#from pycaret.regression import *
pd.set_option('display.max_columns', None)


# Importing the data - don't use *.ML.csv files!!!
If you have *.MLReady.csv then [click here](#MLReady)

In [8]:
df=pd.read_csv("data/FM_Acq2_Perf_FMAC2_loanAge_sample5.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,id,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origDate,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,loanAge,monMatur,mSA,zeroBalCode,deliqGood,deliqBad,deliqMax,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct
0,2008203,2008203,994088840298,B,OTHER,6.500,79000,360,2007-02-01,69.0,69.0,1.0,27.0,R,OH,447,0.0,0.0,633.0,633.0,633.0,77,283.0,15940,1.0,75,2,1.0,6.34,6.22,6.2850,0.019293,4.84,0.160,-0.280,0.2150,0.025237,-0.045016,0.034208
1,1174043,1174043,842724479742,C,"BANK OF AMERICA, N.A.",5.250,140000,360,2003-06-01,49.0,49.0,1.0,31.0,R,CA,956,0.0,0.0,673.0,673.0,673.0,15,345.0,40900,1.0,8,7,6.0,5.26,5.21,5.2300,0.009597,3.37,-0.010,-0.040,0.0200,-0.001901,-0.007678,0.003824
2,460151,460151,870246392264,R,"WITMER FUNDING, LLC",7.375,64000,180,2001-12-01,80.0,80.0,2.0,24.0,C,TX,761,0.0,0.0,615.0,591.0,603.0,60,120.0,19100,1.0,44,16,2.0,7.17,6.84,7.0650,0.048246,4.78,0.205,-0.535,0.3100,0.028591,-0.078216,0.043878
3,1633794,1633794,694595496729,C,OTHER,5.500,90000,360,2004-10-01,97.0,97.0,1.0,46.0,P,IA,525,30.0,1.0,781.0,781.0,781.0,72,288.0,0,9.0,21,50,21.0,5.82,5.64,5.7225,0.031915,4.21,-0.320,0.140,-0.2225,-0.054983,0.024823,-0.038882
4,1151647,1151647,517731724074,R,"JPMORGAN CHASE BANK, NA",5.500,210000,360,2003-06-01,64.0,64.0,2.0,34.0,C,CA,920,0.0,0.0,689.0,689.0,689.0,77,283.0,41740,1.0,73,3,1.0,5.26,5.21,5.2300,0.009597,3.37,0.240,-0.290,0.2700,0.045627,-0.055662,0.051625


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133163 entries, 0 to 133162
Data columns (total 39 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          133163 non-null  int64  
 1   Unnamed: 0.1        133163 non-null  int64  
 2   id                  133163 non-null  int64  
 3   origChannel         133163 non-null  object 
 4   sellerName          133163 non-null  object 
 5   origIntRate         133163 non-null  float64
 6   origUPB             133163 non-null  int64  
 7   origLoanTerm        133163 non-null  int64  
 8   origDate            133163 non-null  object 
 9   origLTV             133163 non-null  float64
 10  origCLTV            133163 non-null  float64
 11  numBorrowers        133163 non-null  float64
 12  origDebtIncRatio    133163 non-null  float64
 13  loanPurp            133163 non-null  object 
 14  propState           133163 non-null  object 
 15  zipCode             133163 non-nul

## Remove columns not involved in 'Foreclosure or Not' query

In [10]:
df.drop(['Unnamed: 0','Unnamed: 0.1','id','fredRate',\
         'bestCreditScore','worstCreditScore','monMatur',\
         'deliqGood','deliqBad','deliqMax',\
        'fmacRateMax','fmacRateMin','fmacRateVolatility',\
        'rateDiffAbove','rateDiffBelow','rateDiffAbovePct','rateDiffBelowPct',\
        ],1,inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133163 entries, 0 to 133162
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   origChannel       133163 non-null  object 
 1   sellerName        133163 non-null  object 
 2   origIntRate       133163 non-null  float64
 3   origUPB           133163 non-null  int64  
 4   origLoanTerm      133163 non-null  int64  
 5   origDate          133163 non-null  object 
 6   origLTV           133163 non-null  float64
 7   origCLTV          133163 non-null  float64
 8   numBorrowers      133163 non-null  float64
 9   origDebtIncRatio  133163 non-null  float64
 10  loanPurp          133163 non-null  object 
 11  propState         133163 non-null  object 
 12  zipCode           133163 non-null  int64  
 13  pMIperct          133163 non-null  float64
 14  mortInsType       133163 non-null  float64
 15  avgCreditScore    133163 non-null  float64
 16  loanAge           13

### Check if any columns have a null value

In [12]:
df.apply(lambda x: x.isnull().sum(), axis=0)

origChannel         0
sellerName          0
origIntRate         0
origUPB             0
origLoanTerm        0
origDate            0
origLTV             0
origCLTV            0
numBorrowers        0
origDebtIncRatio    0
loanPurp            0
propState           0
zipCode             0
pMIperct            0
mortInsType         0
avgCreditScore      0
loanAge             0
mSA                 0
zeroBalCode         0
fmacRateAvg         0
rateDiffAvg         0
rateDiffAvgPct      0
dtype: int64

## Engineer originDate into Date features

In [13]:
df['origDate'] = pd.to_datetime(df['origDate'])

In [14]:
df['origYear'] = df['origDate'].dt.year
df['origMonth'] = df['origDate'].dt.month

In [15]:
df.drop(['origDate'],1,inplace=True)

In [16]:
df[['origYear','origMonth']].head()

,origYear,origMonth
0,2007,2
1,2003,6
2,2001,12
3,2004,10
4,2003,6


# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [17]:
df.zeroBalCode.unique()

array([ 1.,  9.,  6., 16., 15.,  3.,  2.])

In [18]:
df['zeroBalCode'] = df.zeroBalCode.replace([1,6,16],0)
df['zeroBalCode'] = df.zeroBalCode.replace([9,3,2,15],1)

In [19]:
df.zeroBalCode.unique()

array([0., 1.])

In [20]:
#Checking datatypes of individual feature
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,zeroBalCode,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth
0,B,OTHER,6.500,79000,360,69.0,69.0,1.0,27.0,R,OH,447,0.0,0.0,633.0,77,15940,0.0,6.2850,0.2150,0.034208,2007,2
1,C,"BANK OF AMERICA, N.A.",5.250,140000,360,49.0,49.0,1.0,31.0,R,CA,956,0.0,0.0,673.0,15,40900,0.0,5.2300,0.0200,0.003824,2003,6
2,R,"WITMER FUNDING, LLC",7.375,64000,180,80.0,80.0,2.0,24.0,C,TX,761,0.0,0.0,603.0,60,19100,0.0,7.0650,0.3100,0.043878,2001,12
3,C,OTHER,5.500,90000,360,97.0,97.0,1.0,46.0,P,IA,525,30.0,1.0,781.0,72,0,1.0,5.7225,-0.2225,-0.038882,2004,10
4,R,"JPMORGAN CHASE BANK, NA",5.500,210000,360,64.0,64.0,2.0,34.0,C,CA,920,0.0,0.0,689.0,77,41740,0.0,5.2300,0.2700,0.051625,2003,6


# Review values for dates and slice (if needed) the timeframe you want

In [21]:
df.origYear.unique()

array([2007, 2003, 2001, 2004, 2005, 2011, 2002, 2013, 2000, 2006, 2010,
       2012, 2015, 2009, 2008, 2016, 2014, 2018, 1999, 2017, 2019],
      dtype=int64)

In [22]:
df['origMonth'].unique()

array([ 2,  6, 12, 10,  8,  9, 11,  1,  5,  4,  7,  3], dtype=int64)

In [23]:
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,zeroBalCode,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth
0,B,OTHER,6.500,79000,360,69.0,69.0,1.0,27.0,R,OH,447,0.0,0.0,633.0,77,15940,0.0,6.2850,0.2150,0.034208,2007,2
1,C,"BANK OF AMERICA, N.A.",5.250,140000,360,49.0,49.0,1.0,31.0,R,CA,956,0.0,0.0,673.0,15,40900,0.0,5.2300,0.0200,0.003824,2003,6
2,R,"WITMER FUNDING, LLC",7.375,64000,180,80.0,80.0,2.0,24.0,C,TX,761,0.0,0.0,603.0,60,19100,0.0,7.0650,0.3100,0.043878,2001,12
3,C,OTHER,5.500,90000,360,97.0,97.0,1.0,46.0,P,IA,525,30.0,1.0,781.0,72,0,1.0,5.7225,-0.2225,-0.038882,2004,10
4,R,"JPMORGAN CHASE BANK, NA",5.500,210000,360,64.0,64.0,2.0,34.0,C,CA,920,0.0,0.0,689.0,77,41740,0.0,5.2300,0.2700,0.051625,2003,6


# Coerce the column dtype (int in this case)

In [26]:
df = df.astype({'origLTV':'int','origCLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'mortInsType':'int','loanAge':'int',\
               'avgCreditScore':'int','zeroBalCode':'int'})
#,'zeroBalCode':'int'

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133163 entries, 0 to 133162
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   origChannel       133163 non-null  object 
 1   sellerName        133163 non-null  object 
 2   origIntRate       133163 non-null  float64
 3   origUPB           133163 non-null  int64  
 4   origLoanTerm      133163 non-null  int64  
 5   origLTV           133163 non-null  int32  
 6   origCLTV          133163 non-null  int32  
 7   numBorrowers      133163 non-null  int32  
 8   origDebtIncRatio  133163 non-null  int32  
 9   loanPurp          133163 non-null  object 
 10  propState         133163 non-null  object 
 11  zipCode           133163 non-null  int64  
 12  pMIperct          133163 non-null  float64
 13  mortInsType       133163 non-null  int32  
 14  avgCreditScore    133163 non-null  int32  
 15  loanAge           133163 non-null  int32  
 16  mSA               13

<a id='MLReady'></a>

# Oversampling (Since we are starting with only 14.51% of our predicted class)

In [28]:
good = df.zeroBalCode.value_counts()[0]
bad = df.zeroBalCode.value_counts()[1]
perct_bad = round(bad/(good + bad)*100,2)
print(f'We have {perct_bad}% Foreclosures in our dataset')

We have 14.51% Foreclosures in our dataset


In [29]:
from sklearn.model_selection import train_test_split

training_features, test_features, \
training_target, test_target, = train_test_split(
    df.drop(['zeroBalCode'], axis=1)
    , df['zeroBalCode']
    , test_size = .1
    , random_state=12
)

In [30]:
# Further split the training data into training/test
x_train, x_val, y_train, y_val = train_test_split(
    training_features
    , training_target
    , test_size = .1
    ,random_state=12
)

In [31]:
# For the training data, randomly sample 
# ros = RandomOverSampler(sampling_strategy='minority')
# x_train_res, y_train_res = ros.fit_sample(x_train, y_train)

In [32]:
print('#############################################')
print('Before oversampling: "Closed" crushes "Default" and causes issues:')
print(training_target.value_counts())
# print('')
# print('Before oversampling: "Closed" and "Default" are equal')
# print(y_train_res.value_counts())

#############################################
Before oversampling: "Closed" crushes "Default" and causes issues:
0    102437
1     17409
Name: zeroBalCode, dtype: int64


# Recombine X_train_res, y_train_res so PyCaret can deal with one df

In [33]:
y_train.shape

(107861,)

In [34]:
df = x_train.copy()

In [35]:
df['zeroBalCode'] = y_train

In [36]:
df.shape

(107861, 23)

# Review metrics last time before run (to have record of starting values)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107861 entries, 80129 to 48202
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   origChannel       107861 non-null  object 
 1   sellerName        107861 non-null  object 
 2   origIntRate       107861 non-null  float64
 3   origUPB           107861 non-null  int64  
 4   origLoanTerm      107861 non-null  int64  
 5   origLTV           107861 non-null  int32  
 6   origCLTV          107861 non-null  int32  
 7   numBorrowers      107861 non-null  int32  
 8   origDebtIncRatio  107861 non-null  int32  
 9   loanPurp          107861 non-null  object 
 10  propState         107861 non-null  object 
 11  zipCode           107861 non-null  int64  
 12  pMIperct          107861 non-null  float64
 13  mortInsType       107861 non-null  int32  
 14  avgCreditScore    107861 non-null  int32  
 15  loanAge           107861 non-null  int32  
 16  mSA              

In [38]:
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth,zeroBalCode
80129,B,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",7.000,56000,240,73,73,1,20,C,IL,627,0.0,0,624,145,44100,6.068,0.932,0.153593,2002,11,0
1974,C,OTHER,5.375,70000,180,37,37,1,43,C,AZ,850,0.0,0,749,158,38060,6.272,-0.897,-0.143017,2005,12,0
96616,C,OTHER,7.250,119000,360,84,84,1,15,R,TX,762,12.0,1,698,147,0,7.050,0.200,0.028369,2001,2,0
98170,R,"BANK OF AMERICA, N.A.",6.125,66000,240,85,85,1,38,R,AL,356,6.0,1,618,145,22520,5.745,0.380,0.066144,2003,3,0
42259,C,AMTRUST BANK,5.875,184000,180,80,80,1,39,C,WI,548,0.0,0,709,86,0,6.156,-0.281,-0.045647,2007,3,1


# Pycaret - Setup with categorical definition

### Use complex setup 

In [40]:
model_setup = setup(
    df
    , target = 'zeroBalCode' # PyCaret will list this as "Label"
    , normalize = True
    , transformation = False
    , pca = False 
    , ignore_low_variance = False # Variance is calculated using the ratio of unique values to the number of samples, and the ratio of the most common value to the frequency of the second most common value. 
    , ignore_features = None
    , handle_unknown_categorical = True
    , remove_outliers = True
    , remove_multicollinearity = False
    , polynomial_features = False
    , trigonometry_features = False
    , feature_selection = False
    , feature_interaction = False
    , fix_imbalance = True
    , silent = True
    , profile = False
    # outliers from the training data are removed using PCA linear dimensionality reduction using the Singular Value Decomposition technique.
    , bin_numeric_features = [
        'avgCreditScore'
        ] # Set to True to bin numerics using K Means
    , categorical_features = [
         'origChannel'
        , 'loanPurp'
        , 'origYear'
        , 'origMonth'
        , 'mortInsType'
        , 'origLoanTerm'
        ]
    , high_cardinality_features = [
         'propState'
        , 'sellerName'
        , 'mSA'
        , 'zipCode'
    ]
    , high_cardinality_method = 'clustering'
    , numeric_features = [
        'origIntRate'
        , 'numBorrowers'
        , 'origUPB'
        , 'origLTV'
        , 'origCLTV'
        , 'pMIperct'
        , 'loanAge'
        , 'origDebtIncRatio'
        , 'avgCreditScore'
        , 'rateDiffAvg'
        , 'rateDiffAvgPct'
        , 'fmacRateAvg'
    ]
)


Setup Succesfully Completed!


,Description,Value
0,session_id,883
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(107861, 23)"
4,Missing Values,False
5,Numeric Features,14
6,Categorical Features,8
7,Ordinal Features,False
8,High Cardinality Features,True
9,High Cardinality Method,clustering


## Compare Models with no blacklist exclusions

In [41]:
model_results=compare_models(sort='Recall') #blacklist = ['tr','ransac'],'lar','par','huber','llar','lasso','en','ridge','omp','br','svm'])
model_results

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Naive Bayes,0.5508,0.7293,0.8013,0.2182,0.3429,0.1467,0.2192,0.1103
1,SVM - Linear Kernel,0.6895,0.0000,0.7464,0.2859,0.4131,0.2557,0.3103,0.6888
2,Ridge Classifier,0.7060,0.0000,0.7315,0.2959,0.4213,0.2691,0.3186,0.1406
3,Linear Discriminant Analysis,0.7060,0.7825,0.7314,0.2959,0.4213,0.2690,0.3185,1.1173
4,Logistic Regression,0.7101,0.7826,0.7265,0.2985,0.4231,0.2722,0.3202,1.2974
5,K Neighbors Classifier,0.6474,0.7095,0.6825,0.2459,0.3616,0.1866,0.2331,3.0311
6,Quadratic Discriminant Analysis,0.6142,0.6372,0.5825,0.2127,0.3067,0.1197,0.1495,0.5359
7,Ada Boost Classifier,0.7780,0.7734,0.5366,0.3374,0.4142,0.2860,0.2977,9.4679
8,Gradient Boosting Classifier,0.8247,0.7922,0.4309,0.4066,0.4184,0.3153,0.3155,36.2114
9,Decision Tree Classifier,0.7785,0.6128,0.3784,0.2979,0.3333,0.2028,0.2049,2.4856


GaussianNB(priors=None, var_smoothing=1e-09)

## Create models 

### Naive Bayes Classifier

In [42]:
nb = create_model('nb', fold=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5574,0.7227,0.7817,0.2177,0.3406,0.1450,0.2127
1,0.5479,0.7247,0.8017,0.2170,0.3415,0.1446,0.2170
2,0.5553,0.7281,0.7846,0.2173,0.3404,0.1444,0.2127
3,0.5497,0.7300,0.8143,0.2198,0.3462,0.1503,0.2261
4,0.5500,0.7310,0.8124,0.2196,0.3458,0.1498,0.2251
5,0.5569,0.7379,0.8057,0.2215,0.3474,0.1529,0.2267
6,0.5462,0.7388,0.8238,0.2198,0.3470,0.1508,0.2290
7,0.5528,0.7256,0.7798,0.2156,0.3378,0.1409,0.2080
8,0.5510,0.7425,0.8208,0.2212,0.3484,0.1533,0.2310
9,0.5407,0.7116,0.7884,0.2121,0.3343,0.1349,0.2034


## Evaluate Models

In [43]:
evaluate_model(nb)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## Tune Model

In [65]:
tuned_nb = tune_model(nb, optimize = 'Recall')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5574,0.7227,0.7817,0.2177,0.3406,0.1450,0.2127
1,0.5479,0.7247,0.8017,0.2170,0.3415,0.1446,0.2170
2,0.5553,0.7281,0.7846,0.2173,0.3404,0.1444,0.2127
3,0.5497,0.7300,0.8143,0.2198,0.3462,0.1503,0.2261
4,0.5500,0.7310,0.8124,0.2196,0.3458,0.1498,0.2251
5,0.5569,0.7379,0.8057,0.2215,0.3474,0.1529,0.2267
6,0.5462,0.7388,0.8238,0.2198,0.3470,0.1508,0.2290
7,0.5528,0.7256,0.7798,0.2156,0.3378,0.1409,0.2080
8,0.5510,0.7425,0.8208,0.2212,0.3484,0.1533,0.2310
9,0.5407,0.7116,0.7884,0.2121,0.3343,0.1349,0.2034


In [66]:
evaluate_model(tuned_nb)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [58]:
bagged_nb = ensemble_model(nb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5168,0.7348,0.8221,0.2095,0.3340,0.1295,0.2054
1,0.5076,0.7405,0.8335,0.2079,0.3328,0.1269,0.2053
2,0.5114,0.7208,0.8070,0.2053,0.3274,0.1208,0.1919
3,0.5192,0.7382,0.8269,0.2111,0.3363,0.1327,0.2101
4,0.5051,0.7266,0.8127,0.2040,0.3261,0.1184,0.1905
5,0.5266,0.7359,0.8110,0.2116,0.3357,0.1328,0.2062
6,0.5160,0.7409,0.8261,0.2100,0.3349,0.1303,0.2072
7,0.5113,0.7303,0.8108,0.2059,0.3284,0.1220,0.1942
8,0.5151,0.7323,0.8146,0.2078,0.3312,0.1259,0.1994
9,0.5145,0.7240,0.8108,0.2071,0.3299,0.1242,0.1967


In [51]:
boosted_nb = ensemble_model(nb, method='Boosting')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.4744,0.5802,0.6217,0.1636,0.2590,0.0327,0.0505
1,0.6045,0.5373,0.3991,0.1613,0.2297,0.0243,0.0289
2,0.5564,0.4774,0.3594,0.1321,0.1932,-0.0292,-0.0362
3,0.3870,0.5340,0.7104,0.1555,0.2551,0.0167,0.0313
4,0.2146,0.4969,0.9236,0.1499,0.2579,0.0048,0.0189
5,0.3204,0.4577,0.6406,0.1313,0.2179,-0.0363,-0.0749
6,0.4645,0.5437,0.5915,0.1554,0.2461,0.0157,0.0243
7,0.5718,0.5848,0.5156,0.1759,0.2623,0.0540,0.0695
8,0.6845,0.4941,0.1671,0.1136,0.1353,-0.0492,-0.0506
9,0.8359,0.4687,0.0057,0.0462,0.0101,-0.0229,-0.0389


In [ ]:
calibrated_nb = calibrate_model(nb)

IntProgress(value=0, description='Processing: ', max=14)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:45:02
Status,. . . . . . . . . . . . . . . . . .,Fitting Fold 7 of 10
ETC,. . . . . . . . . . . . . . . . . .,25.2 Seconds Remaining


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5302,0.7347,0.8079,0.2124,0.3364,0.1343,0.2072
1,0.5200,0.7405,0.8231,0.2109,0.3357,0.1321,0.2085
2,0.5236,0.7209,0.7871,0.2068,0.3275,0.1227,0.1894
3,0.5352,0.7382,0.8098,0.2146,0.3393,0.1386,0.2123
4,0.5220,0.7265,0.7975,0.2077,0.3296,0.1250,0.1944
5,0.5358,0.7358,0.7902,0.2120,0.3343,0.1325,0.2009


# Predict models

In [44]:
predicted_nb = predict_model(nb)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.5989,0.7482,0.7985,0.2392,0.3681,0.1845,0.2568


# The golden moment... put 10% sample test data through the models...

# Recombine test_features, test_target at first train/test/split
 This "test" set was split BEFORE Oversampling. PyCaret takes the "train" from this split and splits it internally again. Since we have no control of how PyCaret does this we needed to do a train/test split before we input the training set into PyCaret. Now we will recombine the test_features and test_target into one new df

In [45]:
test_features.shape

(13317, 22)

In [46]:
test_target.shape

(13317,)

In [47]:
test_df = test_features.copy()

In [48]:
test_df['zeroBalCode'] = test_target

In [49]:
test_df.shape

(13317, 23)

In [50]:
totalTestCount = test_df.zeroBalCode.value_counts().sum()
posTestCount = test_df.zeroBalCode.value_counts()[1]
print('#############################################')
print('This "Test" set has not been oversamplied')
print(test_df.zeroBalCode.value_counts())
print(f'Positive samples are {round(posTestCount/totalTestCount*100,2)}%')

#############################################
This "Test" set has not been oversamplied
0    11410
1     1907
Name: zeroBalCode, dtype: int64
Positive samples are 14.32%


## Create Confusion Matrix output functions

In [51]:
def calc_confusion(row):
    if ((row['zeroBalCode'] == 0) & (row['Label'] == 0)):
        value = 'TrueNegative'
    elif ((row['zeroBalCode'] == 0) & (row['Label'] == 1)):
        value = 'FalsePositive'
    elif ((row['zeroBalCode'] == 1) & (row['Label'] == 1)):
        value = 'TruePositive'
    elif ((row['zeroBalCode'] == 1) & (row['Label'] == 0)):
        value = 'FalseNegative'
    else:
        value = 'Undefined'
    return value

In [52]:
def cM(df):
    print(f'           ##############################')
    print(f'           #             #              #')
    print(f'         0 #    {df["TrueNegative"]}     #     {df["FalsePositive"]}      #')
    print(f'  True     #             #              #')
    print(f'  Class    ##############################')
    print(f'           #             #              #')
    print(f'         1 #    {df["FalseNegative"]}      #     {df["TruePositive"]}      #')
    print(f'           #             #              #')
    print(f'           ##############################')
    print(f'                  0              1        ')
    print(f'                  Predicted Class           ')


In [53]:
def cMCalc():
    # 0 or 2 = Predict equals truth, 1 = Predict doesn't equal truth
    totalNegative = confusionMatrix['TrueNegative'] + confusionMatrix['FalseNegative']
    totalPositive = confusionMatrix['TruePositive'] + confusionMatrix['FalsePositive']
    total = totalNegative + totalPositive
    trueNegativePct = confusionMatrix['TrueNegative'] / total * 100
    falseNegativePct = confusionMatrix['FalseNegative'] / total * 100
    truePositivePct = confusionMatrix['TruePositive'] / total * 100
    falsePositivePct = confusionMatrix['FalsePositive'] / total * 100

    # Accuracy Calculation
    accuracy = (confusionMatrix['TruePositive'] + confusionMatrix['TrueNegative'])/(totalPositive + totalNegative)
    # Recall Calculation
    recall = (confusionMatrix['TruePositive']/(confusionMatrix['TruePositive'] + confusionMatrix['FalseNegative']))
    # Precision Calculation
    precision = (confusionMatrix['TruePositive']/(confusionMatrix['TruePositive'] + confusionMatrix['FalsePositive']))

    print(f'TrueNegative:  {round(trueNegativePct,2)}%')
    print(f'FalseNegative:  {round(falseNegativePct,2)}%')
    print(f'TruePositive:   {round(truePositivePct,2)}%')
    print(f'FalsePositive: {round(falsePositivePct,2)}%')
    print(f'#####################################################')
    print(f'Accuracy: {round(accuracy,2)*100}%   Recall: {round(recall,2)*100}%   Precision: {round(precision,2)*100}%')
    print(f'#####################################################')

# Evaluate Models

In [54]:
final_nb = finalize_model(nb)

In [55]:
predictions = predict_model(final_nb, data=test_df)

In [56]:
pd.set_option('max_columns',None)
predictions.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth,zeroBalCode,Label,Score
0,B,QUICKEN LOANS INC.,3.750,467000,360,43,43,2,45,R,CA,949,0.0,0,807,81,41860,3.414,0.336,0.098418,2013,1,0,0,0.0000
1,R,OTHER,3.750,74000,360,94,100,1,32,P,OH,458,25.0,1,759,27,0,3.600,0.150,0.041667,2016,5,0,0,0.0150
2,B,"BANK OF AMERICA, N.A.",5.875,108000,360,80,80,1,23,C,WI,531,0.0,0,658,143,0,5.582,0.293,0.052490,2005,6,0,1,0.9321
3,C,"WELLS FARGO BANK, N.A.",5.625,275000,240,72,72,2,34,C,NJ,87,0.0,0,743,163,35620,6.065,-0.440,-0.072547,2005,10,0,0,0.1252
4,R,"BANK OF AMERICA, N.A.",5.625,157000,360,69,69,2,28,R,CA,933,0.0,0,643,120,12540,5.230,0.395,0.075526,2003,6,0,1,0.6400


In [57]:
results = predictions[['zeroBalCode','Label','Score']]

In [58]:
predictions.Label.value_counts()

1    6880
0    6437
Name: Label, dtype: int64

In [59]:
results['Confusion'] = results.apply(calc_confusion, axis=1)

In [60]:
confusionMatrix = results.Confusion.value_counts().to_dict()

In [61]:
cM(confusionMatrix)

           ##############################
           #             #              #
         0 #    5948     #     5462      #
  True     #             #              #
  Class    ##############################
           #             #              #
         1 #    489      #     1418      #
           #             #              #
           ##############################
                  0              1        
                  Predicted Class           


In [62]:
cMCalc()

TrueNegative:  44.66%
FalseNegative:  3.67%
TruePositive:   10.65%
FalsePositive: 41.02%
#####################################################
Accuracy: 55.00000000000001%   Recall: 74.0%   Precision: 21.0%
#####################################################


In [63]:
#save_model(nb, 'np.tuned10pct_091920')

In [64]:
#save_model(calibrated_et, 'extraTreesCalibrated2.5OS5050_082220')